In [2]:
!pip install pandas
!pip install numpy
!pip install requests
!pip install seaborn
!pip install matplotlib


You should consider upgrading via the 'C:\Users\vperezlo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\vperezlo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\vperezlo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\vperezlo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\vperezlo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

In [92]:
mypath = 'C:/Users/vperezlo/OneDrive - Capgemini/Documents/Databases' #Path to your databases folder
write_path = 'C:/Users/vperezlo/OneDrive - Capgemini/Documents/'

url_list = []
data_list = []
data_dict = {}

column_country = 'Country'
column_year = 'Year'
mycolumns = [column_country, column_year]     #Sets the name of the columns we're interested in
columns_rename = dict.fromkeys(['Area', 'Entity', 'Country or Area', 'Name', 'Country Name'], column_country)

indicators = {
    'Armed forces personnel (% of total labor force)': '% Soldiers', 
    'Crude marriage rate (per 1,000 inhabitants)': 'Marriage Rate', 
    'freeexpr_vdem_owid': 'Freedom of Expression', 
    'civlib_vdem_owid': 'Civil Liberties',
    'Indicator:Domestic general government health expenditure (GGHE-D) as percentage of general government expenditure (GGE) (%)': '% Healthcare Investment',
    'Industry as % of total employment -- ILO modelled estimates, May 2017': '% Employment Industry',
    'UIS: Mean years of schooling of the population age 25+. Female': 'Women Schooling Years',
    'UIS: Mean years of schooling of the population age 25+. Male': 'Men Schooling Years',
    'Government expenditure on education, total (% of government expenditure)': '% Education Expenditure',
    'Employment-to-population ratio, men (%)': '% Men Employment',
    'Employment-to-population ratio, women (%)': '% Women Employment',
    'Population (historical estimates)': 'Population',
    '2.a.1 Agriculture value added share of GDP (%)': 'AgriShareGDP',
    'Credit to Agriculture, Forestry and Fishing': 'CreditToAgriFishForest',
    'Employment by status of employment, total, rural areas': 'EmploymentRural',
    'Gross Domestic Product': 'GDP',
    'Share of employment in agriculture, forestry and fishing in total employment': '%EmploymentAgriFishForest',
    'Agriculture': 'TotalAgri',
    'Gender Inequality': 'Gender Inequality',
    'Birth Rate': 'Birth Rate',
    'Value': 'Gini',
    'CPIA gender equality rating (1=low to 6=high)': 'Gender Equality',
    'Intentional homicides (per 100,000 people)': 'Homicides',
    'Prevalence of undernourishment (% of population)': '% Undernourishment'
}

special_source = ['kaggle', 'faostat', 'worldindata', 'worldbank', 'WID', 'databank'] #Delete the ones that need no special

def rename_value_column(dataframe, column_value = None, column_indicator = None, row_index = 1, inplace = False):
    col_values = ['Value']
    col_indicators = ['Item', 'Indicator'] #If no column name specified, iterate over the list of names.
    
    if not column_indicator:
        for indicator in col_indicators:
            if indicator in dataframe:
                column_indicator = indicator
    
    if not column_value:
        for value in col_values:
            if value in dataframe:
                column_value = value             
       
    dataframe.rename(columns = {column_value: dataframe.loc[:, column_indicator][1]}, inplace = inplace)
    return dataframe if not inplace else None

def preprocess (dataframe, treatment = '', melt_on_value = None, rename_value_columns = False):
    
    match treatment:            
        case 'kaggle':            
            dataframe.drop(['HDI Rank'], axis=1, inplace = True)
            melt_on_value = 'Gender Inequality'
            
        case 'faostat':
            rename_value_columns = True
            
        case 'databank':
            
            melt_on_value = dataframe.loc[:, 'Series Name'][1]
            dataframe.drop(['Series Name', 'Series Code', 'Country Code'], axis=1, inplace = True)
        
        case 'worldbank':
            dataframe.drop(['Series Name', 'Series Code', 'Country Code'], axis=1, inplace = True)
            melt_on_value = 'Birth Rate'
            
    dataframe.rename(columns = columns_rename, inplace = True)
    
    if rename_value_columns:
        rename_value_column(dataframe, inplace = True)
    
    if melt_on_value:
        dataframe = pd.melt(dataframe, id_vars=column_country, var_name = column_year, value_name = melt_on_value)
    
    for value in dataframe[column_year]: #Normalize year format
                if type(value) is not int and len(value) > 4:
                    dataframe[column_year].replace({value: str(value[:4])}, inplace = True)
    
    for column in dataframe.columns: #Drop completely empty columns
                if (len(dataframe.loc[:, column].value_counts()) == 1):
                    dataframe.drop(column, axis=1, inplace = True) 
    
    dataframe.rename(columns = indicators, inplace = True)
    dataframe.drop(dataframe.columns.difference(mycolumns + list(indicators.values())), axis = 1, inplace=True)
    return dataframe


#||||||||||START OF MAIN CODE|||||||||||||

#Read all .csv files from the specified folder in mypath
#Add the urls to a url_list, dataframes to a data_list
for element in listdir(mypath):
        url = join(mypath, element)
        if isfile(url) and url.endswith('.csv'):
            url_list.append(url)
            if 'WID' in url:
                continue
            dataframe = pd.read_csv(url)
            special = None
            for source in special_source:
                if source in url.lower():
                    special = source
                    break
            dataframe = preprocess(dataframe, treatment = special)
            data_list.append(dataframe)
            #data_dict[url] = dataframe
    
    
for data in data_list:
    print(data)
    print('\n\n')


              Country  Year Gender Equality
0         Afghanistan  1960              ..
1             Albania  1960              ..
2             Algeria  1960              ..
3      American Samoa  1960              ..
4             Andorra  1960              ..
...               ...   ...             ...
13759             NaN  2021             NaN
13760             NaN  2021             NaN
13761             NaN  2021             NaN
13762             NaN  2021             NaN
13763             NaN  2021             NaN

[13764 rows x 3 columns]



                                                Country  Year  \
0                                           Afghanistan  1960   
1                                           Afghanistan  1960   
2                                               Albania  1960   
3                                               Albania  1960   
4                                               Algeria  1960   
...                                                 .

In [94]:
final_df = data_list[0]

for data in data_list:
    data['Year']= data['Year'].astype(str)
    final_df = pd.merge(final_df, data, on = ["Country", "Year"] , how = "outer")

final_df

C:\Users\vperezlo\AppData\Local\Temp\ipykernel_22444\1941500939.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Birth Rate_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(final_df, data, on = ["Country", "Year"] , how = "outer")
C:\Users\vperezlo\AppData\Local\Temp\ipykernel_22444\1941500939.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Birth Rate_x'} in the result is deprecated and will raise a MergeError in a future version.
  final_df = pd.merge(final_df, data, on = ["Country", "Year"] , how = "outer")


MemoryError: Unable to allocate 3.19 GiB for an array with shape (14, 30546848) and data type object

In [ ]:
final_df.to_csv(r'C:/Users/vperezlo/OneDrive - Capgemini/Documents/TestTable.csv')